In [1]:
#This program tells us how much space each sheet uses.
#For example, sheet1 uses 3MB, sheet2 uses 200KB etc..

In [2]:
def xlsb_sheetnames(file_zip):

    with file_zip.open("xl/workbook.bin") as workbook_bi:
        binary = workbook_bi.read()

    import re
    sheet_number_name_binary_list = re.findall(b'\x00r\x00I\x00d[\s\x000-9]*?.*?[\x9c\x90]', binary)
    
    sheet_names = [(re.sub(b"^[\x000-9]*",b"",fragment.replace(b"\x00r\x00I\x00d",b""))[2:-1].decode(encoding="utf-16")).replace("\x00","") for fragment in sheet_number_name_binary_list]
    return sheet_names

def xlsx_xlsm_sheetnames(file_zip):
    
    from xml.etree import ElementTree
    
    with file_zip.open("xl/workbook.xml") as workbook_xml:
        et = ElementTree.fromstring(workbook_xml.read())
        
    fragments = et.find("{http://schemas.openxmlformats.org/spreadsheetml/2006/main}sheets")
    
    sheet_names = [i.attrib["name"] for i in fragments]
    return sheet_names

def file_size_analysis(filepath):
    import zipfile
    
    file_zip = zipfile.ZipFile(filepath)
    
    sheet_compressed_size = [(i.filename,i.compress_size) for i in file_zip.infolist() if i.filename.startswith("xl/worksheets/") and ("/" not in i.filename.replace("xl/worksheets/","")) and ("Index" not in i.filename)]

    sheet_compressed_size = sorted(sheet_compressed_size, key=lambda x: int(x[0].replace("xl/worksheets/sheet","").replace(".xml","")))

    sheet_compressed_size = [i[1] for i in sheet_compressed_size]
    
    sheet_names = xlsb_sheetnames(file_zip) if filepath.split(".")[-1].lower() == "xlsb" else xlsx_xlsm_sheetnames(file_zip)

    media_files_size = sum([i.compress_size for i in file_zip.infolist() if i.filename.startswith("xl/media/")])

    import pandas as pd
    
    analysis_dictionary = {"Sheet order":range(1,len(sheet_compressed_size)+1),"Sheet name":sheet_names,"File size (compressed)":sheet_compressed_size}
    
    df = pd.DataFrame(analysis_dictionary)

    if media_files_size > 0:
        df2 = pd.DataFrame({"Sheet order":[""],"Sheet name":"[Media files]","File size (compressed)":media_files_size})
        df = pd.concat([df,df2])
        df.reset_index(drop=True,inplace=True)

    df = df.sort_values(by=["File size (compressed)"],ascending = False)
    df = df.assign(**{"Percentage" : lambda x: x["File size (compressed)"]/x["File size (compressed)"].sum()})
    
    df["Percentage"] = df["Percentage"].map(lambda x: "{:.2%}".format(x))
    df["File size (compressed)"] = df["File size (compressed)"].map(lambda x: "{:,}".format(x)).str.replace(","," ")

    return df

In [3]:
file =  r"X:\RI_DATA\RI - Hong Kong\CT HK RI BUSINESS\HK RI MATTERS\RI QUARTER\2024\1Q24\Working files for 24Q1\LAPSE_RI_24_Q1.xlsm"

df = file_size_analysis(file)

display(df)

,Sheet order,Sheet name,File size (compressed),Percentage
7,8,Plan Change History (output),47 913 730,52.72%
3,4,Lapse Tran,10 190 821,11.21%
8,9,RDWDDR CASE,9 901 285,10.90%
9,10,ACT_MthEnd_02.sql,9 312 501,10.25%
10,11,Termination Date,5 023 813,5.53%
2,3,Lapse Tran (For old book),2 958 898,3.26%
4,5,Lapse Tran (For new book),2 303 105,2.53%
6,7,Latest Image(plan change miss),1 647 974,1.81%
0,1,Input,1 520 400,1.67%
5,6,Embedded,98 911,0.11%
